In [ ]:
Freddie Mac dataset with 20 years of mortgage history. The goal is to use H2O-3 to predict the interest rate for each loan, using XGBoost and Deep Learning models. 

In [1]:

# Import libraries 
import os
import h2o
import matplotlib as plt
%matplotlib inline

# Import Estimators
from h2o.estimators.deeplearning import H2ODeepLearningEstimator as H2ODL 
from h2o.estimators import H2OXGBoostEstimator as H2OXGB 
import h2o.grid
from h2o.grid.grid_search import H2OGridSearch

# Start cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.261-b12, mixed mode)
  Starting server from C:\Users\Renzo\AppData\Local\Programs\Python\Python38-32\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Renzo\AppData\Local\Temp\tmp4hgfjrwc
  JVM stdout: C:\Users\Renzo\AppData\Local\Temp\tmp4hgfjrwc\h2o_Renzo_started_from_python.out
  JVM stderr: C:\Users\Renzo\AppData\Local\Temp\tmp4hgfjrwc\h2o_Renzo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_Renzo_ykzqpy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
# Import the dataset
loan_level = h2o.import_file("https://s3.amazonaws.com/data.h2o.ai/DAI-Tutorials/loan_level_500k.csv")

SyntaxError: invalid syntax (&lt;ipython-input-2-b8193dcab7f3&gt;, line 2)